In [1]:
cd /home/disk2/wdd/PDZ/12concentration

/home/disk2/wdd/PDZ/12concentration


In [2]:
import mdtraj as md
import numpy as np

In [3]:
t_ref=md.load('12con_ref.pdb',top='12con_ref.pdb')

/software/anaconda2/envs/msmb/lib/python2.7/site-packages/mdtraj/core/trajectory.py:419: UserWarning: top= kwarg ignored since file contains topology information
  warnings.warn('top= kwarg ignored since file contains topology information')


In [20]:
pdz_heavy_atoms=[atom.index for atom in t_ref.topology.atoms if atom.element.symbol is not 'H'and atom.index <1738]

In [21]:
peptide_heavy_atoms=[atom.index for atom in t_ref.topology.atoms if atom.element.symbol is not 'H'and (atom.residue.chain.index==1)]

In [22]:
atom_pairs=[]
temp=[]
for h in pdz_heavy_atoms:
    for hh in peptide_heavy_atoms:
        temp.append(h)
        temp.append(hh)
        atom_pairs.append(temp)
        temp=[]

In [24]:
len(distances[0])

36666

In [23]:
distances=md.compute_distances(t_ref,atom_pairs=atom_pairs)

In [25]:
native_contacts_index=[]
for i in range(len(atom_pairs)):
    if distances[0][i]<=0.45:
        native_contacts_index.append(atom_pairs[i])

In [35]:
Q(distances_ref[0],distances48[1])

0.99999999999213729

In [27]:
def Q(dist,ref):
    fq_sum=0
    b=5.
    r=1.8
    for i in range(len(dist)):
        fq_sum+=1/(1+np.exp(b*(dist[i]-r*ref[i])))
    return fq_sum/len(dist)
def soft_cut_q1(r, r0, beta=5.0, lambda_constant=1.8):
    r = np.asarray(r)
    r0 = np.asarray(r0)
    result = 1/(1 + np.exp(beta*(r - lambda_constant * r0)))

    return result.sum() / len(r0)

In [28]:
t48=md.load('trj/all/fit48_skip10.xtc',top='begin.pdb')

In [29]:
distances48=md.compute_distances(t48,atom_pairs=native_contacts_index)

In [66]:
for nt in range(48,49):
    print nt
    trjname="trj/all/fit"+str(nt)+"-all.xtc"
    t=md.load(trjname,top='begin.pdb')
    for i in range(1,3):
        all_distances=[]
        all_Qs=[]
        print i
        native_contacts_index=all_nci[i-1]
        dist_trj=md.compute_distances(t,atom_pairs=native_contacts_index)
        distances_ref=md.compute_distances(t_ref,atom_pairs=native_contacts_index)
        filename1='fit'+str(nt)+'_chain'+str(i)+'_distances.cs'
        filename2='fit'+str(nt)+'_chain'+str(i)+'_Qs.cs'
        wf1=open(filename1,'w')
        wf2=open(filename2,'w')
        for ff in range(len(dist_trj)):
            for ll in range(len(dist_trj[ff])):
                wf1.write(str(dist_trj[ff][ll])+'    ')
                wf2.write(str(Qs(dist_trj[ff][ll],distances_ref[0][ll]))+'    ')
            wf1.write('\n')
            wf2.write('\n')
        wf1.close()
        wf2.close()

650

In [30]:
distances_ref=md.compute_distances(t_ref,atom_pairs=native_contacts_index)

In [31]:
len(distances_ref[0])

135

In [6]:
t_ref=md.load('12con_ref.pdb',top='12con_ref.pdb')
pdz_heavy_atoms=[atom.index for atom in t_ref.topology.atoms if atom.element.symbol is not 'H'and atom.index <1738]
def nci(pdz_heavy_atoms,peptide_heavy_atoms):
    atom_pairs=[]
    temp=[]
    for h in pdz_heavy_atoms:
        for hh in peptide_heavy_atoms:
            temp.append(h)
            temp.append(hh)
            atom_pairs.append(temp)
            temp=[]
    distances=md.compute_distances(t_ref,atom_pairs=atom_pairs)
    native_contacts_index=[]
    for i in range(len(atom_pairs)):
        if distances[0][i]<=0.45:
            native_contacts_index.append(atom_pairs[i])
    return native_contacts_index
all_nci=[]
for  i in range(1,13):
    peptide_heavy_atoms=[atom.index for atom in t_ref.topology.atoms if atom.element.symbol is not 'H'and (atom.residue.chain.index==i)]
    native_i=nci(pdz_heavy_atoms,peptide_heavy_atoms)
    all_nci.append(native_i)

In [8]:
def Qs(dist,ref):
    fq_i=[]
    b=5.
    r=1.8
    for i in range(len(dist)):
        fq_i.append(1/(1+np.exp(b*(dist[i]*10-r*ref[i]*10))))
    return fq_i
def m3tom2(sequence,outname):
    import mdtraj.io as mdio
    n_frames = max([len(s) for s in sequence])
    n_trajs = len(sequence)
    new_seq = np.zeros((n_trajs,n_frames))
    new_seq[:] = -1
    for i in range(n_trajs):
        n = len(sequence[i])
        new_seq[i,:n] = sequence[i][:]
    mdio.saveh(outname, new_seq)

for nt in range(2,64):
    print nt
    trjname="trj/all/fit"+str(nt)+"_skip10.xtc"
    t=md.load(trjname,top='begin.pdb')
    for i in range(1,13):
        all_Qs=[]
        print i
        native_contacts_index=all_nci[i-1]
        dist_trj=md.compute_distances(t,atom_pairs=native_contacts_index)
        distances_ref=md.compute_distances(t_ref,atom_pairs=native_contacts_index)
        filename1='/home/disk2/wdd/PDZ/12concentration_MSM_Q/h5/distance/fit'+str(nt)+'_chain'+str(i)+'_distance_s.h5'
        filename2='/home/disk2/wdd/PDZ/12concentration_MSM_Q/h5/Qall/fit'+str(nt)+'_chain'+str(i)+'_Q_s.h5'
        for ff in range(len(dist_trj)):
            all_Qs.append(Qs(dist_trj[ff],distances_ref[0]))
        dist_trj2=np.around(dist_trj,decimals=4)
        all_Qs2=np.around(all_Qs,decimals=4)
        m3tom2(dist_trj2,filename1)
        m3tom2(all_Qs2,filename2)
print "done!!!"

2
1
2
3
4
5
6
7
8
9
10
11
12
3
1
2
3
4
5
6
7
8
9
10
11
12
4
1
2
3
4
5
6
7
8
9
10
11
12
5
1
2
3
4
5
6
7
8
9
10
11
12
6
1
2
3
4
5
6
7
8
9
10
11
12
7
1
2
3
4
5
6
7
8
9
10
11
12
8
1
2
3
4
5
6
7
8
9
10
11
12
9
1
2
3
4
5
6
7
8
9
10
11
12
10
1
2
3
4
5
6
7
8
9
10
11
12
11
1
2
3
4
5
6
7
8
9
10
11
12
12
1
2
3
4
5
6
7
8
9
10
11
12
13
1
2
3
4
5
6
7
8
9
10
11
12
14
1
2
3
4
5
6
7
8
9
10
11
12
15
1
2
3
4
5
6
7
8
9
10
11
12
16
1
2
3
4
5
6
7
8
9
10
11
12
17
1
2
3
4
5
6
7
8
9
10
11
12
18
1
2
3
4
5
6
7
8
9
10
11
12
19
1
2
3
4
5
6
7
8
9
10
11
12
20
1
2
3
4
5
6
7
8
9
10
11
12
21
1
2
3
4
5
6
7
8
9
10
11
12
22
1
2
3
4
5
6
7
8
9
10
11
12
23
1
2
3
4
5
6
7
8
9
10
11
12
24
1
2
3
4
5
6
7
8
9
10
11
12
25
1
2
3
4
5
6
7
8
9
10
11
12
26
1
2
3
4
5
6
7
8
9
10
11
12
27
1
2
3
4
5
6
7
8
9
10
11
12
28
1
2
3
4
5
6
7
8
9
10
11
12
29
1
2
3
4
5
6
7
8
9
10
11
12
30
1
2
3
4
5
6
7
8
9
10
11
12
31
1
2
3
4
5
6
7
8
9
10
11
12
32
1
2
3
4
5
6
7
8
9
10
11
12
33
1
2
3
4
5
6
7
8
9
10
11
12
34
1
2
3
4
5
6
7
8
9
10
11
12
35
1
2
3
4
5
6
7
8

In [9]:
from msmbuilder.dataset import dataset

In [11]:
ds = dataset('/home/disk2/wdd/PDZ/12concentration_MSM_Q/h5/Qall/fit1_chain4_Q_s.h5')

In [12]:
len(ds[0])

10001

In [14]:
ds[0][10000]

array([  1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
         9.98900000e-01,   1.00000000e+00,   1.00000000e+00,
         1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
         1.00000000e+00,   9.99500000e-01,   1.00000000e+00,
         1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
         1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
         1.00000000e+00,   9.99900000e-01,   1.00000000e+00,
         1.00000000e+00,   9.73400000e-01,   1.00000000e+00,
         9.87100000e-01,   9.50800000e-01,   1.00000000e+00,
         1.00000000e+00,   2.81400000e-01,   1.00000000e+00,
         9.75900000e-01,   1.00000000e+00,   9.99300000e-01,
         9.98100000e-01,   1.00000000e+00,   1.00000000e+00,
         1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
         1.00000000e+00,   9.99800000e-01,   9.71700000e-01,
         1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
         1.00000000e+00,   9.99400000e-01,   1.00000000e+00,
         1.00000000e+00,